In [1]:
!pip install python-dotenv gradio openai tiktoken langchain langchain-openai langchain-community langchain-chroma chromadb

Clinical Trial extractions INPUT/MODIFICATION REQUIRED HERE

In [2]:
# -*- coding: utf-8 -*-
"""Simplified Clinical Trials Analyzer with Hardcoded Trial IDs"""

# Imports - keep only what's necessary
import requests
import tiktoken
import gradio as gr
import openai
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# API key and model setup
api_key = #######
openai.api_key = api_key
MODEL = "gpt-3.5-turbo"

# Hardcoded list of clinical trials
CLINICAL_TRIALS = [
    ("NCT04904172", "NCT04904172 - Effectiveness of NIRS at Acute Ischemic Stroke Patients"),
    ("NCT03381248", "NCT03381248 - Cooled Radiofrequency vs. Hyaluronic Acid to Manage Knee Pain"),
    ("NCT04803149", "NCT04803149 - Cooled RFA vs Conventional RFA to Manage Chronic Facetogenic Low Back Pain"),
    ("NCT03601949", "NCT03601949 - Lateral Branch Cooled Radiofrequency Denervation vs. Conservative Therapy for Sacroiliac Joint Pain"),
]

# Cache for storing trial data
trial_cache = {}

# Simplified token management
def truncate_text(text, max_tokens=6000):
    """Truncate text to stay within token limit"""
    try:
        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
        tokens = encoding.encode(text)
        if len(tokens) <= max_tokens:
            return text
        return encoding.decode(tokens[:int(max_tokens*0.9)]) + "... [truncated]"
    except:
        # Fallback to character-based truncation
        char_ratio = 4  # Approx chars per token
        return text[:int(max_tokens*char_ratio*0.9)] + "... [truncated]"

# ClinicalTrials.gov API client
def get_trial_data(nct_id):
    """Get trial data from API or cache"""
    if nct_id in trial_cache:
        return trial_cache[nct_id]

    try:
        response = requests.get(
            f"https://clinicaltrials.gov/api/v2/studies/{nct_id}",
            headers={
                'Accept': 'application/json',
                'User-Agent': 'Mozilla/5.0 (compatible; ResearchBot/1.0)'
            }
        )

        if response.status_code == 200:
            trial_data = response.json()
            trial_cache[nct_id] = trial_data
            return trial_data
        else:
            print(f"Error Response: {response.text}")
            return None
    except Exception as e:
        print(f"Request Error: {e}")
        return None

Setup trial data and organize into question sections

In [3]:

# Extract trial data based on question type
def get_trial_details(trial_id, question_type):
    """Get specific details from a trial based on question type"""
    trial_data = get_trial_data(trial_id)
    if not trial_data:
        return f"Could not retrieve data for trial {trial_id}"

    # Protocol section shortcuts
    protocol = trial_data.get("protocolSection", {})
    ident = protocol.get("identificationModule", {})
    status = protocol.get("statusModule", {})
    eligibility = protocol.get("eligibilityModule", {})
    sponsors = protocol.get("sponsorCollaboratorsModule", {})
    conditions = protocol.get("conditionsModule", {})
    description = protocol.get("descriptionModule", {})

    # Results section shortcuts
    results = trial_data.get("resultsSection", {})
    outcomes = results.get("outcomeMeasuresModule", {})
    adverse = results.get("adverseEventsModule", {})

    # Handle different question types
    if question_type == "summary":
        details = f"# Summary of Trial {trial_id}\n\n"
        details += f"**Title**: {ident.get('briefTitle', 'Unknown')}\n"
        details += f"**Status**: {status.get('overallStatus', 'Unknown')}\n"
        details += f"**Sponsor**: {sponsors.get('leadSponsor', {}).get('name', 'Unknown')}\n"
        details += f"**Start Date**: {status.get('startDateStruct', {}).get('date', 'Unknown')}\n"
        details += f"**Completion Date**: {status.get('completionDateStruct', {}).get('date', 'Unknown')}\n"

        trial_conditions = conditions.get("conditions", [])
        if trial_conditions:
            details += f"**Conditions**: {', '.join(trial_conditions)}\n"

        details += f"\n**Brief Summary**: {description.get('briefSummary', 'No summary available')}\n"

        return details

    elif question_type == "eligibility":
        criteria = eligibility.get("eligibilityCriteria", "No eligibility criteria found")
        return f"# Eligibility Criteria for Trial {trial_id}\n\n{criteria}"

    elif question_type == "outcomes":
        details = f"# Outcome Measures for Trial {trial_id}\n\n"

        outcome_list = outcomes.get("outcomeMeasures", [])
        if outcome_list:
            for i, outcome in enumerate(outcome_list, 1):
                details += f"## {i}. {outcome.get('title', 'Unknown')} ({outcome.get('type', 'Unknown')})\n"
                details += f"**Description**: {outcome.get('description', 'No description')}\n"
                details += f"**Time Frame**: {outcome.get('timeFrame', 'Unknown')}\n\n"
        else:
            details += "No outcome measures found for this trial."

        return details

    elif question_type == "adverse_events":
        details = f"# Adverse Events for Trial {trial_id}\n\n"

        if adverse:
            if "seriousEvents" in adverse:
                details += "## Serious Adverse Events\n\n"
                for event in adverse["seriousEvents"]:
                    details += f"- **{event.get('term', 'Unknown')}**: "
                    if "stats" in event:
                        for stat in event["stats"]:
                            group_id = stat.get("groupId", "Unknown")
                            affected = stat.get("numAffected", 0)
                            at_risk = stat.get("numAtRisk", 0)
                            details += f"Group {group_id}: {affected}/{at_risk} participants affected. "
                    details += "\n"

            if "otherEvents" in adverse:
                details += "\n## Other Adverse Events\n\n"
                for event in adverse["otherEvents"]:
                    details += f"- **{event.get('term', 'Unknown')}**: "
                    if "stats" in event:
                        for stat in event["stats"]:
                            group_id = stat.get("groupId", "Unknown")
                            affected = stat.get("numAffected", 0)
                            at_risk = stat.get("numAtRisk", 0)
                            details += f"Group {group_id}: {affected}/{at_risk} participants affected. "
                    details += "\n"
        else:
            details = f"No adverse events information found for Trial {trial_id}."

        return details

    elif question_type == "custom":
        # For custom questions, prepare a simplified data structure for the LLM
        return {
            "id": trial_id,
            "title": ident.get("briefTitle", "Unknown"),
            "official_title": ident.get("officialTitle", "Unknown"),
            "status": status.get("overallStatus", "Unknown"),
            "sponsor": sponsors.get("leadSponsor", {}).get("name", "Unknown"),
            "brief_summary": description.get("briefSummary", "Unknown"),
            "conditions": conditions.get("conditions", []),
            "eligibility": eligibility.get("eligibilityCriteria", ""),
            "primary_outcomes": [
                {"title": o.get("title", "Unknown"), "description": o.get("description", "No description")}
                for o in outcomes.get("outcomeMeasures", []) if o.get("type", "") == "PRIMARY"
            ]
        }

    else:
        return f"Unknown question type: {question_type}"


setup LLM

In [4]:

# Set up LLM
llm = ChatOpenAI(openai_api_key=api_key, temperature=0.7, model_name=MODEL, max_tokens=1000)

# Consolidated chain creation function
def create_question_chain(question_type):
    """Create a chain for a specific question type"""
    templates = {
        "summary": """
            You are an expert in clinical trials and medical research.
            Below is information about a clinical trial.

            {details}

            Provide a concise summary of this clinical trial. Focus on:
            - The purpose of the trial
            - The status (completed, recruiting, etc.)
            - Key points about conditions being studied
            - The sponsor

            YOUR RESPONSE:
        """,
        "eligibility": """
            You are an expert in clinical trials and medical research.
            Below are the eligibility criteria for a clinical trial.

            {details}

            Summarize the key inclusion and exclusion criteria in a clear, organized way.
            Make this easy for a patient to understand if they might qualify.

            YOUR RESPONSE:
        """,
        "outcomes": """
            You are an expert in clinical trials and medical research.
            Below are the outcome measures for a clinical trial.

            {details}

            Summarize the key outcome measures and what they tell us about the trial results.
            Distinguish between primary and secondary outcomes where possible.

            YOUR RESPONSE:
        """,
        "adverse_events": """
            You are an expert in clinical trials and medical research.
            Below is information about adverse events in a clinical trial.

            {details}

            Summarize the key safety findings from this trial:
            - What serious adverse events occurred?
            - How common were they?
            - Are there any safety concerns that stand out?

            YOUR RESPONSE:
        """,
        "custom": """
            You are an expert in clinical trials and medical research.
            Below is information about a clinical trial and a custom question about it.

            TRIAL INFORMATION:
            {details}

            USER QUESTION: {question}

            Answer the user's question based on the trial information provided.
            Be specific and cite the relevant parts of the trial information.
            If the information needed to answer the question is not provided, state that clearly.

            YOUR RESPONSE:
        """
    }

    template = templates.get(question_type, templates["summary"])
    variables = ["details"]
    if question_type == "custom":
        variables.append("question")

    prompt = PromptTemplate(input_variables=variables, template=template)
    return LLMChain(llm=llm, prompt=prompt)

# Main function to answer questions about trials
def answer_trial_question(trial_id, question_type, custom_question=None):
    """Generate answers about clinical trials based on question type"""
    details = get_trial_details(trial_id, question_type)

    # If details is a string, it's already formatted for display
    if isinstance(details, str) and question_type != "custom":
        return details

    # Create and run the appropriate chain
    chain = create_question_chain(question_type)

    if question_type == "custom":
        # Format details for custom questions
        details_str = "\n".join([f"{key.upper()}: {value}" for key, value in details.items()])
        response = chain.invoke({"details": details_str, "question": custom_question or "Tell me about this trial"})
    else:
        response = chain.invoke({"details": details})

    return response["text"]


Gradio Interface

In [5]:

# Gradio interface
def create_interface():
    question_types = ["summary", "eligibility", "outcomes", "adverse_events", "custom"]

    with gr.Blocks(title="Clinical Trial Navigator") as interface:
        gr.Markdown("# Clinical Trial Navigator")
        gr.Markdown("Select a clinical trial and explore the data")

        with gr.Row():
            with gr.Column(scale=3):
                # Trial selection dropdown
                trial_dropdown = gr.Dropdown(
                    choices=[trial[1] for trial in CLINICAL_TRIALS],
                    value=CLINICAL_TRIALS[0][1] if CLINICAL_TRIALS else None,
                    label="Select a Clinical Trial"
                )

                # Question type selection
                question_type = gr.Radio(
                    choices=question_types,
                    label="Question Type",
                    value="summary"
                )

                # Custom question input
                custom_question = gr.Textbox(
                    lines=2,
                    label="Custom Question",
                    placeholder="Enter your specific question...",
                    visible=False
                )

                # Make custom question visible only when custom is selected
                question_type.change(
                    fn=lambda q_type: gr.update(visible=(q_type == "custom")),
                    inputs=[question_type],
                    outputs=[custom_question]
                )

                # Submit button
                submit_btn = gr.Button("Get Answer")

            with gr.Column(scale=5):
                # Output area
                output = gr.Markdown(label="Answer")

        # Handle submission
        submit_btn.click(
            fn=lambda trial_selection, q_type, custom_q: (
                answer_trial_question(
                    trial_selection.split(" - ")[0],
                    q_type,
                    custom_q
                ) if trial_selection else "Please select a trial"
            ),
            inputs=[trial_dropdown, question_type, custom_question],
            outputs=[output]
        )

    return interface

# Launch the interface
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://45c42fffba5ede5d1e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-4-61365a14c459>:80: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://45c42fffba5ede5d1e.gradio.live
